### Máquina de Turing Não Determinística (MTND)

In [1]:
from typing import Dict, List, DefaultDict, Optional

from collections import defaultdict

In [2]:
#Função de desenvolvimento

DEBUG_LOG : bool = False

def debugPrint(*args):
  if DEBUG_LOG : print(' '.join([str(value) for value in args]))

In [3]:
# states = input() #"0 1 2 3 4"
# symbols = input() #"a b"
# P_symbols = input() #"A B *"

# F_limit = input() #"<"
# F_blank = input() #"*"

# n_transitions= int(input()) #3
# transitions= [] #"0 a * 0 A", "0 * * 1 *", "1 b A 1 *"

# for i in range(0, n_transitions):
#   transitions.append(input())

# initial_state= input() #"0"
# final_state= input() #"1"
# words= input() #"* ab ba abb aab"

In [4]:
states = "0 1 2 3 4"
symbols = "a b"
P_symbols = "A B *"
F_limit = "<"
F_blank = "*"
n_transitions= 10
transitions=["0 a 1 A D", "1 a 1 a D", "1 B 1 B D", "1 b 2 B E", "2 B 2 B E", "2 a 2 a E", "2 A 0 A D", "0 B 3 B D", "3 B 3 B D", "3 * 4 * E"]

initial_state= "0"
final_state= "4"
words= "* ab ba abb aab aabb"

In [5]:
class Edge:
  symbol       : str
  write        : str
  destination  : Optional["State"]
  direction    : int

  def __init__(self, symbol: str, write: str, destination, direction: str):
    self.symbol      = symbol
    self.destination = destination
    self.write       = write
    self.direction   = direction

class StackStates:
  tape        : str
  state       : Optional["State"]
  belongs     : True
  indice      : int

  def __init__(self, tape: str, state, belongs = False, indice = 0):
    self.state   = state
    self.tape   = tape
    self.belongs = belongs
    self.indice  = indice

class State:
  transitions : DefaultDict[str, List[Edge]]
  isFinal     : bool
  state       : str

  def __init__(self, state):
    self.state       = state
    self.transitions = defaultdict(list)
    self.isFinal     = False

  def addTransition(self, symbol: str, write: str, destination: Optional["State"], direction: int) -> None:
    self.transitions[symbol].append(Edge(symbol, write, destination, direction))

  def becomeFinalState(self) -> None:
    self.isFinal = not self.isFinal

class MTND:
  stateList     : List[State]
  initialState  : State

  def __init__(self, stateList: List[State]):
    self.stateList = stateList

  def getState(self, state: str) -> State:
    return [s for s in self.stateList if s.state == state][0]

  def becomeInitialState(self, initial: str):
    self.initialState = self.getState(initial)
  
  def checkWord(self, word: str) -> str:
    stackStates : List[StackStates] = [StackStates("<"+word, self.initialState, indice=1)] # Inicia a lista com a pilha vazia e o estado atual o inicial

    # debugPrint(word)
    # debugPrint(apnd.initialState.state, [""])

    while [s for s in stackStates if not s.belongs]:
      nextStackStates : List[StackStates] = []

      #Navega na dimensão de estados
      for ss in stackStates:
        debugPrint("Current:",[(s.tape, s.indice, s.state.state, s.belongs) for s in [ss]])
        if(ss.belongs) : 
          debugPrint("Pertence a Linguagem")
          break
        
        #armazena todas as possiveis transições de uma dimensão
        edge = ss.state.transitions[ss.tape[ss.indice] if (len(ss.tape) > ss.indice) else "*"].copy()
          
        debugPrint("Finding:", ss.tape, ss.indice, [(e.symbol, e.destination.state, e.destination.isFinal) for e in edge])

        #Percorra para cada caminho encontrado
        for e in edge:
          nexttape = ss.tape

          if(len(ss.tape) - 1 < ss.indice): 
            nexttape = nexttape + "*"*(len(nexttape) - ss.indice + 1)
          
          debugPrint("Pilha: ", nexttape, "Escreva: ", e.write, "em ", ss.indice )
          
          #Se ouver um caminho e o item da pilha for desempilhavel, desempilhe
          if (nexttape and (nexttape[ss.indice] == e.symbol)): 
            nexttape = (nexttape[:ss.indice] + e.write + nexttape[ss.indice+1:])
            nextindex = ss.indice + e.direction

            nextbelongs = not e.destination.transitions[ss.tape[nextindex] if (len(ss.tape) > nextindex) else "*"] and e.destination.isFinal
            
            if nextbelongs:
              return "S"
            else:
              nextStackStates.append(StackStates(nexttape, e.destination, nextbelongs, nextindex))

            debugPrint("For:", e.symbol)
            debugPrint(e.destination.state, [nexttape])

      debugPrint("Next:", [(s.tape, s.state.state, s.tape, s.indice, s.belongs) for s in nextStackStates])
    
      if len(nextStackStates) > 0:
        stackStates = nextStackStates
      else:
        stackStates = []
        break

    if ([s for s in stackStates if s.belongs]):
      return "S"
    else:
      return "N"

In [6]:
#Functions
def processStates(states: List[str]):
  """
  Retorna uma lista de States a partir de uma lista de estados.
  """
  return [State(s) for s in states]

def processTrasitions(transitions: List[str], mtnd: MTND):
  """
  Liga os estados de uma lista de trasições a outros estados a partir de um caminho nomeado.
  """
  for o, c, d, w, e in [trn.split(" ") for trn in transitions]:
    st = mtnd.getState(o)
    dir = -1 if e == "E" else (1 if e == "D" else 0)
    st.addTransition(c, w, mtnd.getState(d), dir)

def processFinalStates(final_states: List[str], mtnd: MTND):
  """
  Torna todos os itens de uma lista de estados finais.
  """
  for f in final_states.split(" "):
    s = mtnd.getState(f)
    s.becomeFinalState()

def processWords(words: List[str], mtnd: MTND):
  """
  Verifica se as palavras de uma lista pertencem ao MTND.
  """
  for w in words.split(" "):
    print(mtnd.checkWord(w))

In [7]:

mtnd = MTND(processStates(states.split(" ")))
debugPrint(transitions)
processTrasitions(transitions, mtnd)
mtnd.becomeInitialState(initial_state)
processFinalStates(final_state, mtnd)
processWords(words, mtnd)

N
S
N
N
N
S


In [10]:
import time

wa = 'a'
wb = 'b'


for i in range(50):
  w = wa+wb
  
  inicio = time.time()

  result = mtnd.checkWord(w)
  
  fim = time.time()
  print(str(result)+ " "+str(len(w))+ "	" + str(fim - inicio).replace('.',','))

  wa = wa*2
  wb = wb*2

S 2	6,818771362304688e-05
S 4	0,00014543533325195312
S 8	0,0003540515899658203
S 16	0,0012576580047607422
S 32	0,004567384719848633
S 64	0,017350196838378906
S 128	0,043532371520996094
S 256	0,17992639541625977
S 512	0,7574450969696045
S 1024	2,940030097961426
S 2048	11,820150375366211
S 4096	48,800758838653564
S 8192	203,18213319778442
S 16384	889,4777178764343


KeyboardInterrupt: ignored